# Day 9: Disk Fragmenter

### Part 1

Another push of the button leaves you in the familiar hallways of some friendly amphipods! Good thing you each somehow got your own personal mini submarine. The Historians jet away in search of the Chief, mostly by driving directly into walls.

While The Historians quickly figure out how to pilot these things, you notice an amphipod in the corner struggling with his computer. He's trying to make more contiguous free space by compacting all of the files, but his program isn't working; you offer to help.

He shows you the disk map (your puzzle input) he's already generated. For example:

```
2333133121414131402
```

The disk map uses a dense format to represent the layout of files and free space on the disk. The digits alternate between indicating the length of a file and the length of free space.

So, a disk map like 12345 would represent a one-block file, two blocks of free space, a three-block file, four blocks of free space, and then a five-block file. A disk map like 90909 would represent three nine-block files in a row (with no free space between them).

Each file on disk also has an ID number based on the order of the files as they appear before they are rearranged, starting with ID 0. So, the disk map 12345 has three files: a one-block file with ID 0, a three-block file with ID 1, and a five-block file with ID 2. Using one character for each block where digits are the file ID and . is free space, the disk map 12345 represents these individual blocks:

```
0..111....22222
```

The first example above, `2333133121414131402`, represents these individual blocks:

00...111...2...333.44.5555.6666.777.888899
The amphipod would like to move file blocks one at a time from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks). For the disk map 12345, the process looks like this:

```
0..111....22222
02.111....2222.
022111....222..
0221112...22...
02211122..2....
022111222......
```
The first example requires a few more steps:

```
00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
```

The final step of this file-compacting process is to update the filesystem checksum. To calculate the checksum, add up the result of multiplying each of these blocks' position with the file ID number it contains. The leftmost block is in position 0. If a block contains free space, skip it instead.

Continuing the first example, the first few blocks' position multiplied by its file ID number are 0 * 0 = 0, 1 * 0 = 0, 2 * 9 = 18, 3 * 9 = 27, 4 * 8 = 32, and so on. In this example, the checksum is the sum of these, 1928.

Compact the amphipod's hard drive using the process he requested. What is the resulting filesystem checksum? (Be careful copy/pasting the input for this puzzle; it is a single, very long line.)




In [1]:
import com.psyndicate.aoc.LinkedList
import com.psyndicate.aoc.readResourceAsText
import java.util.*
import kotlin.test.assertEquals

val testInput = "2333133121414131402"

data class Block(val id: Int, var used: Int, var free: Int) {
    fun fitsInside(other: Block) = used <= other.free

    override fun toString(): String = "ID: $id Used:$used Free:$free"

    companion object {
        fun parse(input: String): LinkedList<Block> {
            val blocks = LinkedList<Block>()
            for (index in 0 until input.length step 2) {
                val newBlock = Block(
                    id = index / 2,
                    used = input[index].digitToInt(),
                    free = if (index + 1 < input.length) input[index + 1].digitToInt() else 0
                )
                blocks.append(newBlock)
            }
            return blocks
        }
    }
}

fun LinkedList<Block>.compact(): LinkedList<Block> {
    var head: LinkedList.Node<Block>? = this.first
    var tail: LinkedList.Node<Block>? = this.last

    while (head != tail) {
        head?.value?.let { h ->
            while (h.free > 0) {
                tail?.value?.let { t ->
                    val usage = if (t.used >= h.free) h.free else t.used
                    insertAfter(head!!, Block(id = t.id, used = usage, free = h.free - usage))
                    h.free = 0
                    t.used -= usage
                    t.free += usage
                    if (t.used == 0) {
                        tail?.let {
                            tail = it.previous
                            remove(it)
                        }
                    }
                }
            }
        }
        head = head?.next
    }
    return this
}

fun LinkedList<Block>.checksum(): Long {
    var position = 0
    return iterator().sumOf { block ->
        var sum = 0L
        for (i in 0 until block.used) {
            sum = sum + position * block.id
            position++
        }
        position += block.free
        sum
    }
}

fun calculatePart1(input: String): Long = Block.parse(input).compact().checksum()

val testInputPart1Result = calculatePart1(testInput)
println("Test Part 1: ${testInputPart1Result}")
assertEquals(1928L, testInputPart1Result)

val puzzleInput = readResourceAsText("/day9/input.txt")
val part1Result = calculatePart1(puzzleInput)
println("Part 1: ${part1Result}")
assertEquals(6337367222422, part1Result)

Test Part 1: 1928
Part 1: 6337367222422


### Part 2

Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that file system fragmentation was a bad idea?

The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving whole files instead.

This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of decreasing file ID number starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.

The first example from above now proceeds differently:
```
00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
```

The process of updating the filesystem checksum is the same; now, this example's checksum would be 2858.

Start over, now compacting the amphipod's hard drive using this new method instead. What is the resulting filesystem checksum?



In [2]:
fun LinkedList.Node<Block>.moveAfter(other: LinkedList.Node<Block>) {
        // Block previous to this block free increased by used + free of this block
        this.previous?.let { it.value.free += value.used + value.free }

        // This block free = other - this
        this.value.free = other.value.free - this.value.used

        // Remove self from list
        previous?.next = next
        next?.previous = previous

        // Insert self after other (not considering first & last ptrs)
        next = other.next
        previous = other
        other.next?.previous = this
        other.next = this

        // Other block space consumed, free space is now in this block
        other.value.free = 0
}

fun LinkedList<Block>.compactWholeFiles(): LinkedList<Block> {
    var candidate = last
    outer@while(candidate != null) {
        var ptr : LinkedList.Node<Block>? = this.first
        while(ptr != null) {
            if (ptr == candidate) break
            if (candidate?.value?.fitsInside(ptr.value) == true) {
                val save = candidate.previous
                candidate.moveAfter(ptr)
                candidate = save
                continue@outer
            }
            ptr = ptr.next
        }
        candidate = candidate?.previous
    }
    return this
}

fun calculatePart2(input: String): Long = Block.parse(input).compactWholeFiles().checksum()

val testInputPart2Result = calculatePart2(testInput)
println("Test Part 2: ${testInputPart2Result}")
assertEquals(2858, testInputPart2Result)

val part2Result = calculatePart2(puzzleInput)
println("Part 2: ${part2Result}")
assertEquals(6361380647183L, part2Result)

Test Part 2: 2858
Part 2: 6361380647183
